# 1. Importing the dataset

In [ ]:
!pip install pmdarima # for arima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade -q gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

In [ ]:
gc = gspread.authorize(creds)

# importing the sales data
data = gc.open_by_url('https://docs.google.com/spreadsheets/d/1o2lQ_-zMDYdflRfYDUnXq7g1Jxem3fXeW4l_2MeUsVU/edit?usp=sharing')
worksheet = data.worksheet('Copy of Products Sold') # Enter your sheet name.

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df=pd.DataFrame.from_records(rows)

new_header = df.iloc[1] #grab the second row for the header
df = df[2:] #take the data less the header row
df.columns = new_header #set the header row as the df header

df.head(10)

[['', '', 'Markets', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['Products', 'To forecast?', 'Unit sold', '2022-06-20', '2022-06-06', '2022-05-13', '2022-04-22', '2022-03-25', '2022-02-11', '2022-01-14', '2021-12-17', '2021-11-26', '2021-10-22', '2021-09-24', '2021-08-27', '2021-08-13', '2021-07-23', '2021-07-09'], ['Apples', 'Yes', 'unit', '', '209', '734', '395', '713', '595', '562', '557', '527', '715', '699', '544', '329', '515', '497'], ['Bananas', 'No', 'unit', '', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['Beans', 'Yes', 'scoop', '', '0', '0', '0', '8', '8', '0', '0', '4', '3', '0', '6', '0', '8', '6'], ['Beets', 'Yes', 'unit', '', '0', '117', '0', '63', '95', '0', '0', '82', '77', '60', '52', '3', '64', '48'], ['Broccoli', 'Yes', 'unit', '', '0', '77.22', '0', '37', '25', '0', '0', '34', '6', '41', '61', '23', '62', '62'], ['Brussels Sprouts', 'No', '', '', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['C

1,Products,To forecast?,Unit sold,2022-06-20,2022-06-06,2022-05-13,2022-04-22,2022-03-25,2022-02-11,2022-01-14,2021-12-17,2021-11-26,2021-10-22,2021-09-24,2021-08-27,2021-08-13,2021-07-23,2021-07-09
2,Apples,Yes,unit,,209,734,395,713,595,562,557,527,715,699,544,329,515,497
3,Bananas,No,unit,,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Beans,Yes,scoop,,0,0,0,8,8,0,0,4,3,0,6,0,8,6
5,Beets,Yes,unit,,0,117,0,63,95,0,0,82,77,60,52,3,64,48
6,Broccoli,Yes,unit,,0,77.22,0,37,25,0,0,34,6,41,61,23,62,62
7,Brussels Sprouts,No,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Cabbage,Yes,unit,,0,0,0,0,16,0,42,14,28,29,1,19,1,41
9,Carrots,Yes,unit,,171,1186,1011,1208,984,647,975,617,517,776,668,695,1117,958
10,Cauliflower,No,unit,,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,Celery,Yes,unit,,37,13,29,49,33,17,38,13,15,31,47,21,42,34


In [ ]:
# importing the external data for the models
data = gc.open_by_url('https://docs.google.com/spreadsheets/d/1o2lQ_-zMDYdflRfYDUnXq7g1Jxem3fXeW4l_2MeUsVU/edit?usp=sharing')
worksheet = data.worksheet('External data') # Enter your sheet name.

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
ex=pd.DataFrame.from_records(rows)

new_header = ex.iloc[0] #grab the second row for the header
ex = ex[1:] #take the data less the header row
ex.columns = new_header #set the header row as the df header

ex

[['Date of the Market', 'temperature', 'precipitation', 'Apples', 'Bananas', 'Beans', 'Beets', 'Broccoli', 'Brussels Sprouts', 'Cabbage', 'Carrots', 'Cauliflower', 'Celery', 'Chickpeas', 'Chives', 'Cucumber', 'Eggs', 'Garlic', 'Ginger', 'Green beans', 'Honey', 'Lentils', 'Milk', 'Mushrooms', 'Oats', 'Onions', 'Oranges', 'Peas', 'Potatoes', 'Radish', 'Rice', 'Rutabaga', 'Salad', 'Spinach', 'Sprouts', 'Squash', 'Sugar', 'Tomatoes'], ['2021-07-09', '18.5', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'], ['2021-07-23', '21.4', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'], ['2021-08-13', '26.7', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1',

,Date of the Market,temperature,precipitation,Apples,Bananas,Beans,Beets,Broccoli,Brussels Sprouts,Cabbage,...,Potatoes,Radish,Rice,Rutabaga,Salad,Spinach,Sprouts,Squash,Sugar,Tomatoes
1,2021-07-09,18.5,0,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
2,2021-07-23,21.4,0,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
3,2021-08-13,26.7,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,2021-08-27,21.3,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,2021-09-24,18.4,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,2021-10-22,8.9,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,2021-11-26,0.7,1,1,1,0,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
8,2021-12-17,4.7,0,1,1,0,1,0,1,1,...,1,0,1,1,0,1,1,1,1,0
9,2022-01-14,-14.8,0,1,1,0,1,0,1,1,...,1,0,1,1,0,0,1,1,1,0
10,2022-02-11,2.6,1,1,1,0,1,0,1,1,...,1,0,1,1,0,0,1,1,1,0


In [ ]:
# importing the the seasonal guide to use within forecast predictions for random forest and linear regression
data = gc.open_by_url('https://docs.google.com/spreadsheets/d/1o2lQ_-zMDYdflRfYDUnXq7g1Jxem3fXeW4l_2MeUsVU/edit?usp=sharing')
worksheet = data.worksheet('seasonal produce guide') # Enter your sheet name.

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
spg=pd.DataFrame.from_records(rows)

new_header = spg.iloc[0] #grab the second row for the header
spg = spg[1:] #take the data less the header row
spg.columns = new_header #set the header row as the df header

spg

[['', 'apples', 'asparagus', 'bananas', 'beans', 'beets', 'bell peppers', 'blueberries', 'broccoli', 'brussels sprouts', 'cabbage', 'cantaloupe', 'carrots', 'cauliflower', 'celeriac', 'celery', 'chickpeas', 'chives', 'corn', 'cranberries', 'cucumber', 'eggs', 'eggplant', 'endive', 'fennel', 'fresh herbs', 'garlic', 'ginger', 'grapes', 'green beans', 'green onions', 'ground cherries', 'honey', 'jerusalem artichoke', 'kale', 'leeks', 'lentils', 'salad', 'milk', 'mushrooms', 'oats', 'onions', 'oranges', 'parsnip', 'pears', 'peas', 'potatoes', 'pumpkin', 'radish', 'raspberries', 'rhubarb', 'rice', 'rutabaga', 'snow peas', 'spinach', 'sprouts', 'squash', 'strawberries', 'sugar', 'sweet potatoes', 'swiss chard', 'tomatoes', 'turnip', 'watermelon', 'zucchini'], ['January', '1', '0', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0'

,,apples,asparagus,bananas,beans,beets,bell peppers,blueberries,broccoli,brussels sprouts,...,sprouts,squash,strawberries,sugar,sweet potatoes,swiss chard,tomatoes,turnip,watermelon,zucchini
1,January,1,0,1,0,1,0,0,0,1,...,1,1,0,1,1,0,0,1,0,0
2,February,1,0,1,0,1,0,0,0,1,...,1,1,0,1,1,0,0,1,0,0
3,March,1,0,1,0,1,0,0,0,1,...,1,0,0,1,1,0,1,0,0,0
4,April,1,0,1,0,1,0,0,0,0,...,1,0,0,1,1,0,1,0,0,0
5,May,1,1,1,0,1,0,0,0,0,...,1,0,0,1,1,0,1,0,0,0
6,June,1,1,1,0,1,0,0,0,0,...,1,1,1,1,1,1,1,1,0,0
7,July,1,1,1,1,1,1,0,1,0,...,1,1,1,1,1,1,1,1,0,1
8,August,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,September,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10,October,1,0,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,0


# 2. Data cleaning/formatting

In [ ]:
df2=df[df["To forecast?"]=="Yes"] # selecting only the products you want to forecast
df2

1,Products,To forecast?,Unit sold,2022-06-20,2022-06-06,2022-05-13,2022-04-22,2022-03-25,2022-02-11,2022-01-14,2021-12-17,2021-11-26,2021-10-22,2021-09-24,2021-08-27,2021-08-13,2021-07-23,2021-07-09
2,Apples,Yes,unit,,209,734,395,713,595,562,557,527,715,699,544,329,515,497
4,Beans,Yes,scoop,,0,0,0,8,8,0,0,4,3,0,6,0,8,6
5,Beets,Yes,unit,,0,117,0,63,95,0,0,82,77,60,52,3,64,48
6,Broccoli,Yes,unit,,0,77.22,0,37,25,0,0,34,6,41,61,23,62,62
8,Cabbage,Yes,unit,,0,0,0,0,16,0,42,14,28,29,1,19,1,41
9,Carrots,Yes,unit,,171,1186,1011,1208,984,647,975,617,517,776,668,695,1117,958
11,Celery,Yes,unit,,37,13,29,49,33,17,38,13,15,31,47,21,42,34
12,Chickpeas,Yes,scoop,,8,14,20,16,8,0,0,6,8,0,4,15,13,18
14,Cucumber,Yes,unit,,75,36,0,0,0,0,0,69,51,33,44,53,27,61
15,Eggs,Yes,dozen,,56,112,157,162,254,284,50,91,79,217,252,66,70,208


In [ ]:
df3=df2.drop(columns=["To forecast?", "Unit sold",	"2022-06-20"]) # dropping columns that are not needed for forecasting
df3.head()

1,Products,2022-06-06,2022-05-13,2022-04-22,2022-03-25,2022-02-11,2022-01-14,2021-12-17,2021-11-26,2021-10-22,2021-09-24,2021-08-27,2021-08-13,2021-07-23,2021-07-09
2,Apples,209,734,395,713,595,562,557,527,715,699,544,329,515,497
4,Beans,0,0,0,8,8,0,0,4,3,0,6,0,8,6
5,Beets,0,117,0,63,95,0,0,82,77,60,52,3,64,48
6,Broccoli,0,77.22,0,37,25,0,0,34,6,41,61,23,62,62
8,Cabbage,0,0,0,0,16,0,42,14,28,29,1,19,1,41


In [ ]:
ex=ex.set_index("Date of the Market")

In [ ]:
ex.temperature=ex.temperature.astype(float) # converting from object to float
ex.iloc[:,1:]=ex.iloc[:,1:].astype(int) # converting from object to int

In [ ]:
ex

,temperature,precipitation,Apples,Bananas,Beans,Beets,Broccoli,Brussels Sprouts,Cabbage,Carrots,...,Potatoes,Radish,Rice,Rutabaga,Salad,Spinach,Sprouts,Squash,Sugar,Tomatoes
Date of the Market,,,,,,,,,,,,,,,,,,,,,
2021-07-09,18.5,0,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2021-07-23,21.4,0,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2021-08-13,26.7,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2021-08-27,21.3,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2021-09-24,18.4,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2021-10-22,8.9,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2021-11-26,0.7,1,1,1,0,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
2021-12-17,4.7,0,1,1,0,1,0,1,1,1,...,1,0,1,1,0,1,1,1,1,0
2022-01-14,-14.8,0,1,1,0,1,0,1,1,1,...,1,0,1,1,0,0,1,1,1,0


In [ ]:
# have to include this within the front-end design for user input
weather=float(input("Forecasted temperature:"))
precip=int(input("Will there be precipitation (rain/snow) (1=yes, 0=no)?:"))
month=int(input("What is the month of the next market? (1=January, 2=February, 3=March, ...)"))

Forecasted temperature:25.3
Will there be precipitation (rain/snow) (1=yes, 0=no)?:0
What is the month of the next market? (1=January, 2=February, 3=March, ...)6


#3. Demand forecast models

In [ ]:
numproducts=len(df3) # number of products to forecast

In [ ]:
import statsmodels.api as sm # for moving average
from statsmodels.tsa.arima_model import ARIMA 
import pmdarima as pm # for auto arima
from statsmodels.tsa.holtwinters import SimpleExpSmoothing # for simple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing # for holts winters exponential smoothing
from sklearn.ensemble import RandomForestRegressor # for the random forest
import numpy as np # for the time variable in the regression
from sklearn.linear_model import LinearRegression # for linear regression

from sklearn.metrics import mean_absolute_percentage_error # for model assessment
from sklearn.metrics import mean_squared_error # for model assessment
from math import sqrt

In [ ]:
products={} # define empty dictionary to store predictions
train_test=0.9 # train

# model competition to produce the most accurate forecast - accuracy based on MAPE as it is an easy to comprehend measure of accuracy for the end users
for i in range(numproducts):

  # restructuring to have univariate time series format
  df4=pd.DataFrame(df3.iloc[i,:])
  product=df4.iloc[0,0] # get the product name

  df4["Date"]=df4.index # renaming the index as the date
  df4=df4[1:]
  df4.columns=["Sales","Date"] # renaming the columns to be date and sales
  df4 = df4.reset_index(drop=True) # dropping the index values to ensure the date is just the index
  df4=df4.sort_values(by="Date") # putting the dates from oldest to newest order
  df4=df4.set_index("Date")
  df4["Sales"]=df4["Sales"].astype(float) # have to convert the sales into a number type, will keep it as float in case there happens to be any decimal values


  # splitting the dataset into train and test for the model competition
  train_num=int(round(len(df4)*train_test,0)) #can discuss how much we want to split it for train and test
  train=df4.iloc[:train_num,:]
  test=df4.iloc[train_num:,:]

  #-----------------------------------------------------------

  # Model 0 = moving average
  mode_ma=sm.tsa.ARIMA(train,order=(0,0,1))
  model_fit_ma = mode_ma.fit()

  ma = pd.DataFrame(model_fit_ma.predict(start=len(train)+1, end=len(df4)))
  ma.columns=["MA"]
  ma=ma.reset_index(drop=True)

  # Model 1 = auto arima
  model_aa=pm.auto_arima(df4, start_p=1, start_q=1, 
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=12,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

  aa = pd.DataFrame(model_aa.predict(len(test)))
  aa.columns=["AA"]
  aa=aa.reset_index(drop=True)


  # Model 2 = simple exponential smoothing
  model_se=SimpleExpSmoothing(df4)
  model_fit_se=model_se.fit()
  se = pd.DataFrame(model_fit_se.predict(start=len(train)+1, end=len(df4)))
  se.columns=["SE"]
  se=se.reset_index(drop=True)


  # Model 3 = holt winter's exponential smoothing
  model_he=ExponentialSmoothing(df4)
  model_fit_he=model_he.fit()
  he = pd.DataFrame(model_fit_he.predict(start=len(train)+1, end=len(df4)))
  he.columns=["HE"]
  he=he.reset_index(drop=True)


  # Model 4 = naive forecasting - taking the previous period's actuals as the forecast
  na = pd.DataFrame(train.Sales[len(train)-1].repeat(len(test)))
  na.columns=["NA"]
  na=na.reset_index(drop=True)


  #------------------ SUPERVISED MODELS----------------------------

  inseason=int(spg[product.lower()][month])

  df5=df4
  temp=ex[["temperature", "precipitation"]]
  df6=pd.concat([df5,temp,ex[product]], axis = 1)

  df6["Sales-1"]=df6.Sales.shift(1)
  df6["Sales-2"]=df6.Sales.shift(2)
  df6["Sales-3"]=df6.Sales.shift(3)
  df6=df6.dropna() # have to drop na values from the shift or else the model won't run

  # splitting the dataset into train and test for the model competition
  train_num=int(round(len(df6)*train_test,0)) #can discuss how much we want to split it for train and test
  train=df6.iloc[:train_num,:]
  X_train=train.iloc[:,1:]
  y_train=train["Sales"]

  test=df6.iloc[train_num:,:]
  X_test=test.iloc[:,1:]
  y_test=test["Sales"]


  # Model 5 = Random Forest
  rf = RandomForestRegressor(n_estimators=100, random_state=1)
  rf_fit=rf.fit(X_train,y_train)

  rf = pd.DataFrame(rf_fit.predict(X_test))
  rf.columns=["RF"]
  rf=rf.reset_index(drop=True)



  # Model 6 = Linear regression
  df7=df6
  df7["time"]=np.arange(len(df7.index))

  train_num=int(round(len(df7)*train_test,0)) #can discuss how much we want to split it for train and test
  train=df7.iloc[:train_num,:]
  X_train=train.iloc[:,1:]
  y_train=train["Sales"]

  test=df7.iloc[train_num:,:]
  X_test=test.iloc[:,1:]
  y_test=test["Sales"]


  lr = LinearRegression()
  lr_fit=lr.fit(X_train, y_train)

  lr = pd.DataFrame(lr_fit.predict(X_test))
  lr.columns=["LR"]
  lr=lr.reset_index(drop=True)


  # Assesing the best model using MAPE
  predictions=pd.concat([test.reset_index(), ma,aa,se,he,na,rf,lr], axis=1)

  rmse=[sqrt(mean_squared_error(predictions["Sales"],predictions["MA"])),
        sqrt(mean_squared_error(predictions["Sales"],predictions["AA"])),
        sqrt(mean_squared_error(predictions["Sales"],predictions["SE"])),
        sqrt(mean_squared_error(predictions["Sales"],predictions["HE"])),
        sqrt(mean_squared_error(predictions["Sales"],predictions["NA"])),
        sqrt(mean_squared_error(predictions["Sales"],predictions["RF"])),
        sqrt(mean_squared_error(predictions["Sales"],predictions["LR"]))]
  
  """mape=[mean_absolute_percentage_error(predictions["Sales"],predictions["MA"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["AA"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["SE"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["HE"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["NA"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["RF"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["LR"])]"""
  
  best_model=rmse.index(min(rmse)) # selecting the index of the best model


  #------------------------------------------------------------

  # Predicting the value for the next market based on the most accurate model

  # If the best model is the moving average
  if best_model==0:
    forecast = int(model_fit_ma.predict(len(df4)))

  # If the best model is the auto arima
  elif best_model==1:
    forecast = int(model_aa.predict(1))

  # If the best model is the simple exponential smoothing
  elif best_model==2:
    forecast = int(model_fit_se.predict(len(df4)))

  # If the best model is the Holt's winter's exponential smoothing
  elif best_model==3:
    forecast = int(model_fit_he.predict(len(df4)))

  # If the best model is naive
  elif best_model==4:
    forecast=int(df4.Sales[len(df4)-1])

  # If the best model is the random forest
  elif best_model==5:
    forecast = int(rf_fit.predict([[weather,precip,inseason,df6["Sales"][-1],df6["Sales"][-2],df6["Sales"][-3]]]))

  # If the best model if the regression
  else:
    forecast = int(lr_fit.predict([[weather,precip,inseason,df7["Sales"][-1],df7["Sales"][-2],df7["Sales"][-3],len(df7)]]))


  # storing the best forecasted value in the dictionary - to be used within the optimization model
  products[product]= 0 if forecast<0 else forecast # if there's a model that produces a negative value, then we convert it to 0 


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-pac

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.30 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=180.153, Time=0.02 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=171.732, Time=0.10 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=178.328, Time=0.02 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=172.819, Time=0.20 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.44 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=169.748, Time=0.08 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=170.861, Time=0.19 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=167.875, Time=0.18 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=inf, Time=0.13 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=inf, Time=0.36 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=inf, Time=0.27 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AIC=inf, Time=0.31 sec
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=inf, Time=0.28 sec

Best model:  

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: Us

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.32 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=88.772, Time=0.02 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=85.495, Time=0.08 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=86.777, Time=0.02 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=84.490, Time=0.13 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=83.484, Time=0.19 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=81.511, Time=0.20 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=82.507, Time=0.10 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=inf, Time=0.38 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=inf, Time=0.08 sec

Best model:  ARIMA(3,2,0)(0,0,0)[0]          
Total fit time: 2.353 seconds


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/ts

...

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: Us

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=103.483, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=104.091, Time=0.04 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=101.553, Time=0.01 sec

Best model:  ARIMA(0,2,0)(0,0,0)[0]          
Total fit time: 0.300 seconds


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/ts

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=89.595, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=86.943, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=87.757, Time=0.03 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=85.524, Time=0.04 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=87.521, Time=0.05 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=87.151, Time=0.08 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.18 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=83.527, Time=0.03 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=84.975, Time=0.02 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=85.524, Time=0.03 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=inf, Time=0.08 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=inf, Time=0.20 sec

Best model:  A

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/ts

In [ ]:
products # forecasted values for the optimizer, these are in the units in which they are sold at the market, pls refer to the conversion tab in the google sheet "Invoice Information"

{'Apples': 828,
 'Beans': 0,
 'Beets': 120,
 'Broccoli': 34,
 'Cabbage': 0,
 'Carrots': 212,
 'Celery': 30,
 'Chickpeas': 8,
 'Cucumber': 149,
 'Eggs': 0,
 'Garlic': 212,
 'Ginger': 38,
 'Lentils': 2,
 'Milk': 150,
 'Onions': 30,
 'Oranges': 191,
 'Potatoes': 215,
 'Rice': 21,
 'Salad': 0,
 'Spinach': 22,
 'Sprouts': 0,
 'Squash': 15}

#... optimizer...